# Assignment

In [16]:
#!pip install nltk

In [231]:
import pandas as pd
import numpy as np
import os
import sklearn
np.set_printoptions(threshold=sys.maxsize)
from sklearn.model_selection import train_test_split
import re
from glob import glob
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [232]:
nltk.download('stopwords')
nltk.download('punkt')
#print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [237]:
path = "./sample_50/"

In [233]:
def get_authors_count(soup):
    author_count = 0
    
    contrib = soup.find('contrib-group')
    if contrib is None:
        return author_count
    
    authors = contrib.find_all('contrib', **{'contrib-type':"author"})
    for author in authors:
        name = author.find('name')
        if name is not None:
            #author_list.append(' '.join([author.find('surname').text, author.find('given-names').text]))
            author_count += 1

    return author_count

In [101]:
def get_abstract(soup):
    if soup.find("abstract") is None:
        return "NaN"
    
    return soup.find('abstract').text

In [234]:
def get_reference(soup):
    reference_count = 0
    
    contrib = soup.find('ref-list')
    if contrib is None:
        #print("here")
        return reference_count
    
    reference_count = len(contrib.find_all('ref'))
    
    #print(reference_count)
    return reference_count

In [235]:
def get_section(soup):
    reference_count = 0
    
    contrib = soup.find('body')
    if contrib is None:
        #print("here")
        return reference_count
    
    reference_count = len(contrib.find_all('sec'))
    
    #print(reference_count)
    return reference_count

In [236]:
from pathlib import Path
import glob
from bs4 import BeautifulSoup

In [238]:
def get_sample_data(path):
    pathlist = Path(path).glob('**/*.xml')
    abstracts,labels = [],[]
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        soup = BeautifulSoup(open('./'+ path_in_str, 'r'))
        labels.append(get_authors_count(soup))
        abstracts.append(get_abstract(soup))
#         reference.append(get_reference(soup))
#         sections.append(get_section(soup))

    return np.array(abstracts), np.array(labels)


In [246]:
X, y = get_sample_data(path)
y = [1 if each >=5 else 0 for each in y]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


## Assignment1

1) build classification models using Bag-Of-Words (unigram) features from abstracts only, and making the labels the # of authors (This task doesn’t make sense at all, but as a practice, just try). To make it classification (not regression), make it binary “many” (equal or more than 5 authors) and “few” (less than 5 authors). For classification algorithms, use logistic regression and SVM. 

In [120]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [121]:
# print("vocab size: {}".format(len(count_vect.vocabulary_)))
# print("vocab content:\n {}".format(count_vect.vocabulary_))

vocab size: 2584
vocab content:
 {'community': 548, 'acquired': 172, 'lower': 1418, 'respiratory': 2020, 'tract': 2393, 'infections': 1248, 'lrti': 1422, 'and': 266, 'pneumonia': 1763, 'cap': 430, 'are': 290, 'common': 543, 'causes': 459, 'of': 1626, 'morbidity': 1533, 'mortality': 1536, 'among': 252, 'those': 2365, 'aged': 222, '65': 112, 'years': 2579, 'growing': 1097, 'population': 1773, 'in': 1202, 'many': 1444, 'countries': 651, 'detailed': 734, 'incidence': 1207, 'estimates': 906, 'for': 1028, 'these': 2353, 'older': 1629, 'adults': 208, 'the': 2338, 'united': 2449, 'kingdom': 1340, 'uk': 2430, 'lacking': 1351, 'we': 2530, 'used': 2470, 'electronic': 848, 'general': 1068, 'practice': 1788, 'records': 1950, 'from': 1048, 'clinical': 504, 'research': 2010, 'data': 680, 'link': 1397, 'linked': 1399, 'to': 2377, 'hospital': 1145, 'episode': 899, 'statistics': 2233, 'inpatient': 1261, 'estimate': 905, 'between': 384, 'april': 288, '1997': 35, 'march': 1446, '2011': 48, 'by': 416, 'age

In [240]:
X_trans = count_vect.transform(X)

In [241]:
X_trans.shape

(50, 2584)

In [139]:
# print(X_trans)
# print("bag of words as an array: {}".format(bag_of_words.toarray()))

In [243]:
# count_vect.get_feature_names()

### Train test split

In [247]:
X_train, X_test, y_train, y_test = train_test_split(X_trans, y, test_size = 0.2, random_state = 0)
print(X_test.shape[0], len(y_test))  # sparse matrix

10 10


### SVM classifier

In [220]:
from sklearn.linear_model import SGDClassifier # by default, it fits a linear support vector machine (SVM).

In [248]:
svm_model = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)
svm_model.fit(X_train, y_train)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [251]:
pred_svm = svm_model.predict(X_test)
np.mean(pred_svm == y_test)

0.8

### Logistics Regression

In [129]:
from sklearn.linear_model import LogisticRegression

In [252]:
logistic_model = LogisticRegression(C = 100,random_state = 0)
logistic_model.fit(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [258]:
predicted_logistic = logistic_model.predict(X_test)
np.mean(predicted_logistic == y_test)

0.6

## Assignment 2

2) build SVM and logistic models for the same task as (1) with all features including Bag-Of-Words (unigram) 
features from the abstracts, the number of references the paper include, and the number of sections. 
make the label (classification goal) as “highly cited’ and “not cited much’. The threshold for deciding 
the two classes can be anything. Just pick an arbitrary number that can divide the articles into two classes.

In [149]:
#y_binary = [1 if each >=5 else 0 for each in y]

In [259]:
def get_sample_data_2(path):
    pathlist = Path(path).glob('**/*.xml')
    reference, section = [], []
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        soup = BeautifulSoup(open('./'+ path_in_str, 'r'))
        section.append(get_section(soup))
        reference.append(get_reference(soup))
    return np.array(reference)[..., None], np.array(section)[..., None]

In [260]:
reference_count, section_count = get_sample_data_2(path)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [261]:
reference_count.shape, section_count.shape

((50, 1), (50, 1))

In [262]:
X_trans.shape

(50, 2584)

In [263]:
X_trans = np.hstack([X_trans.toarray(), reference_count, section_count])

## Train Test split

In [269]:
X_train, X_test, y_train, y_test = train_test_split(X_trans, y, test_size = 0.2, random_state = 0)
print(X_test.shape[0], len(y_test))

10 10


### SVM classifier

In [272]:
svm_model = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)
svm_model.fit(X_train, y_train)

# predicted_svm = text_clf_svm.predict(twenty_test.data)
# np.mean(predicted_svm == twenty_test.target)


SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [273]:
pred_svm = svm_model.predict(X_test)
np.mean(pred_svm == y_test)

0.9

### Logistic classifier

In [274]:
logistic_model = LogisticRegression(C = 100,random_state = 0)
logistic_model.fit(X_train, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [275]:
predicted_logistic = logistic_model.predict(X_test)
np.mean(predicted_logistic == y_test)

0.8

# TODO

- train-val-test: 0.8, 0.1, 0.1
- at least 1000 data sample instead of 50

In [268]:
# example_sent = "This is a sample sentence, showing off the stop words filtration." 
# stop_words = set(stopwords.words('english')) 
# word_tokens = word_tokenize(example_sent) 
# filtered_sentence = [w for w in word_tokens if not w in stop_words] 
# print(word_tokens) 
# print(filtered_sentence)

In [ ]:
# def tokenize(sentences):
#     words = []
#     for sentence in sentences:
#         w = word_extraction(sentence)
#         words.extend(w)
#     print("tokenize")
#     print(words)
#     words = sorted(list(set(words)))
#     return words

# def word_extraction(sentence):
#     ignore = ['a', "the", "is"]
#     words = re.sub("[^\w]", " ",  sentence).split()
# #     print("word_extraction")
# #     print(words)
#     cleaned_text = [w.lower() for w in words if w not in ignore]
# #     print("cleaned_text")
# #     print(cleaned_text)
#     return cleaned_text    
    
# def generate_bow(allsentences):    
#     vocab = tokenize(allsentences)
#     print("Word List for Document \n{0} \n".format(vocab));

#     for sentence in allsentences:
#         words = word_extraction(sentence)
#         bag_vector = np.zeros(len(vocab))
#         for w in words:
#             for i,word in enumerate(vocab):
#                 if word == w: 
#                     bag_vector[i] += 1
                    
#         print("{0} \n{1}\n".format(sentence,numpy.array(bag_vector)))

# # allsentences = ["Joe waited for the train", "The train was late", "Mary and Samantha took the bus", 
# #             "I looked for Mary and Samantha at the bus station", 
# #             "Mary and Samantha arrived at the bus station early but waited until noon for the bus"]
# allsentences = ["Joe waited for the train", "The train was late"]

# generate_bow(allsentences)